## Punto 1.1

In [4]:
import bs4 # pip install 
from bs4 import BeautifulSoup

In [5]:
import requests
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'
result = requests.get(url)
result.text

'\r\n\r\n\r\n\r\n<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html lang="en" class="no-js ie ie6 ieFail"> <![endif]-->\n<!--[if IE 7 ]> <html lang="en" class="no-js ie ie7 ieFail"> <![endif]-->\n<!--[if IE 8 ]> <html lang="en" class="no-js ie ie8"> <![endif]-->\n<!--[if IE 9 ]> <html lang="en" class="no-js ie ie9"> <![endif]-->\n<!--[if IE 9 ]> <html lang="en" class="no-js ie ie9"> <![endif]-->\n<!--[if !(IE)]><!--> <html lang="en" class="no-ie no-js"> <!--<![endif]-->\n<head><title>\r\n\tMSc Degrees\r\n</title><meta name="referrer" content="always" /><link rel="amphtml" href="https://www.findamasters.com/masters-degrees/courses-amp_v2.aspx?catid=01Mf40&amp=true" />\n<link rel="canonical" href="https://www.findamasters.com/masters-degrees/msc-degrees/" />\n<meta name="description" content="FindAMasters. Search Postgraduate MSc Degrees Worldwide." /><meta name="keywords" content="MSc Degrees" /><link rel="next" href="/masters-degrees/msc-degrees/?PG=2" />\n<script src="/common/js/jQuery/jquery.

In [7]:
def extract_masters(this_url):
    result_url=requests.get(this_url)
    result_soup=BeautifulSoup(result_url.text)
    result_links=result_soup.find_all('a',{'class':'courseLink'})
    result_list=[]
    for item in result_links:
        result_list.append((item['href'],item.text))
    return result_list

In [8]:
# to extract all the masters we have to do a for loop for the first 400 pages:
"""
num_pages = 400
pref = 'https://www.findamasters.com'
test_lst_all = []

for i in range(1, num_pages + 1):
    test_lst = extract_masters(pref + '/masters-degrees/msc-degrees/?PG=' + str(i))
    test_lst_all.extend(test_lst)
# creating the txt file of the fisrt 400 pages of ms
with open('masters_urls.txt', 'w') as f:
    for item in test_lst_all:
        f.write(item[0] + '\n')
"""
# file created 

"\nnum_pages = 400\npref = 'https://www.findamasters.com'\ntest_lst_all = []\n\nfor i in range(1, num_pages + 1):\n    test_lst = extract_masters(pref + '/masters-degrees/msc-degrees/?PG=' + str(i))\n    test_lst_all.extend(test_lst)\n# creating the txt file of the fisrt 400 pages of ms\nwith open('masters_urls.txt', 'w') as f:\n    for item in test_lst_all:\n        f.write(item[0] + '\n')\n"

## Punto 1.2

In [12]:
import os
import requests
from bs4 import BeautifulSoup
import time
import random

In [13]:
# Impostazioni per l'User-Agent per simulare un browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
folder_name = "html_pages"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i in range(1, 4):
    # Crea la cartella per ogni pagina:
    name = 'HTML page ' + str(i)
    path_folder = os.path.join(folder_name, name)
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)

# Apre il file contenente gli URL
with open('masters_urls.txt', 'r') as file:
    for index, url in enumerate(file):
        url = url.strip()
        page = (index // 15 ) + 1
        try:
            # Costruisci l'URL completo
            full_url = "https://www.findamasters.com" + url.strip()

            # Aggiungi un ritardo di 1-5 secondi tra le richieste
            time.sleep(1 + random.uniform(0, 4))
            # Richiesta per ottenere il contenuto dell'URL
            response = requests.get(full_url, headers=headers)
            
            if response.status_code == 200:
                # Parsing dell'HTML con BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')
            
                # Salva l'HTML del corso in un file separato nella cartella della pagina in cui si trova
                file_path = os.path.join(f"{folder_name}\HTML page {page}", f"course {index+1}.html")
                with open(file_path, "w", encoding="utf-8") as html_file:
                    html_file.write(str(soup))

            else:
                print(f"Error while retrieving the course page: {full_url}")
        
        except Exception as e:
            print(f"Error during retrieval and saving of the page {full_url}: {e}")


## Punto 1.3

In [ ]:
#A questo punto io dal 1.2 avrò una cartella per ogni pagina chiamata "HTML page i" (i=1, ..., 400) e ognuna conterrà 15 file HTML.
#Gli HTML sono in formato ".html".

In [1]:
#Questa funzione prende il file .html di ogni pagina di un master, msc_page_url, ed estrapola le informazioni richieste.

from bs4 import BeautifulSoup

def extract_msc_page(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        contents = []

        page_soup = BeautifulSoup(file, 'html.parser')

        course_containers = page_soup.find_all('div', class_='course-header')

        for course_container in course_containers:
            course_info = {}

            # Course Name
            name_links = course_container.find_all('a', class_='course-header__course-title')
            if name_links:
                course_info['courseName'] = name_links[0].text.strip()
            else:
                course_info['courseName'] = ''

            # University Name
            university_links = course_container.find_all('a', class_='course-header__institution')
            if university_links:
                course_info['universityName'] = university_links[0].text.strip()
            else:
                course_info['universityName'] = ''

            # Faculty Name (department)
            faculty_links = course_container.find_all('a', class_='course-header__department')
            if faculty_links:
                course_info['facultyName'] = faculty_links[0].text.strip()
            else:
                course_info['facultyName'] = ''

            # Full or Part Time
            full_time_links = course_container.find_all('a', class_='concealLink')
            fullTime = False
            for item in full_time_links:
                if item['href'] == '/masters-degrees/full-time/':
                    fullTime = True
                    break
            course_info['isItFullTime'] = fullTime

            # Short Description
            description = course_container.find('div', {'id': 'Snippet'}).text.strip()
            course_info['description'] = description

            # Start Date, Fees, Modality, Duration, City, Country, Presence or online modality, Link to the page.
            key_value_pairs = course_container.find_all('div', class_='course-intro__key-info__item')
            for pair in key_value_pairs:
                key = pair.find('div', class_='course-intro__key-info__label').text.strip()
                value = pair.find('div', class_='course-intro__key-info__value').text.strip()
                course_info[key] = value

            contents.append(course_info)

        return contents

In [9]:
import os
import pandas as pd

#Directory where there are the HTML repositories
html_folder = r"C:\Users\sofyc\OneDrive\Desktop\Homework3_adm\html_pages"
#Devo mettere il mio percorso qui al posto di "html_pages" ??
#"C:\Users\sofyc\OneDrive\Desktop\Homework3_adm\html_pages"

#List to contain all the information
all_master_info = []

#Iterating whitin the repositories HTML of every page
for page_folder in os.listdir(html_folder):
    page_path = os.path.join(html_folder, page_folder)
    if os.path.isdir(page_path):
        #Iterating in the files HTML of every repository
        for file in os.listdir(page_path):
            if file.endswith(".html"):
                file_path = os.path.join(page_path, file) #Taking the full path for each HTML file within the current folder
                #Applying the function extract_msc_page to every file HTML
                master_info = extract_msc_page(file_path)
                all_master_info.extend(master_info)

#Creating the DataFrame
df = pd.DataFrame(all_master_info)

#Visualizing the DataFrame
print(df)

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [ ]:
#For each master's degree, creating a course_i.tsv file:

# Output directory for TSV files
output_dir = "C:\Users\sofyc\OneDrive\Desktop\Homework3_adm"  #Questa è la mia directory chiamata Homework3_adm

#Creating the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
#Spiegazione: check to ensure that the directory is available for saving the TSV files. 
# #If the directory already exists, it has no effect, and the program continues to execute without errors.

#Iterating through the DataFrame and creating a TSV file for each master's degree
for index, row in df.iterrows():
    tsv_filename = f"course_{index}.tsv"
    tsv_path = os.path.join(output_dir, tsv_filename)
    
    with open(tsv_path, "w", encoding="utf-8") as tsv_file:
        # Write the information to the TSV file
        tsv_file.write(f"{row['courseName']}\t{row['universityName']}\t{row['facultyName']}\t{row['isItFullTime']}\t{row['description']}\t{row['startDate']}\t{row['fees']}\t{row['modality']}\t{row['duration']}\t{row['city']}\t{row['country']}\t{row['administration']}\t{row['url']}\n")